In [1]:
import os
import glob 
import xlwt
import datetime
import pandas as pd 
from datetime import date
from PyPDF2 import PdfFileReader

In [2]:
def cosmonth_change(mon):
    MONTH=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV']
    mon=mon.replace(" ","")
    if MONTH.index(mon[2:5])+l < 10:
        mon=mon.replace(f'{mon[2:5]}' ,f'0{MONTH,index(mon[2:5])+l}')
        year=mon[4:]
        date=mon[:2]
        month=mon[2:4]
        mon=year + "/" + month + "/" + date
    else:
        mon=mon.replace(f'{mon[2:5]}',f'{MONTH. index(mon[2:5])+l}')
        year=mon[4:]
        date= mon[:2]
        month=mon[2:4]
        mon= year + "/" + month + "/" + date
    return mon

In [3]:
def setStyle(fname="Times New Roman",colour=1,fcolour=0,fheight=200,bold=False,horz=0x02,vert=0x01):
    style=xlwt.XFStyle()
    
    #字體樣式
    font=xlwt.Font()
    font.name=fname
    font.colour_index=fcolour
    font.bold=bold
    style.font=font

    #單元格顏色樣式設定
    pattern=xlwt.Pattern()
    pattern.pattern=xlwt.Pattern.SOLID_PATTERN
    pattern.pattern_fore_colour=colour
    style.pattern=pattern

    #邊框樣式設定
    border=xlwt.Borders()
    border.left=0x05
    border.top=0x05
    border.right=0x05
    border.bottom=0x05
    style.borders=border
    
    #對齊樣式設定
    alignment=xlwt.Alignment()
    alignment.horz=horz
    alignment.vert=vert
    style.alignment=alignment
    return style

In [4]:
# 找到擺放不同broker商的交易單檔案路徑
with open (r'SPOT&SWAP PATH.txt', "r",encoding="utf-8") as f:
    f=f.read()
    f=f.split("\n")
    
path= []
for p in f:
    ky=p.split("=")
    print(ky)
    path.append(ky)
    print("---------------------split---------------------")
path=dict(path)

#把今天日期紀錄起來
today=date.today()
today=today.strftime('%Y%m%d')

['COSP', 'C:\\Users\\USER\\Desktop\\星展實習程式\\*.pdf']
---------------------split---------------------
['COSW', 'C:\\Users\\USER\\Desktop\\星展實習程式\\*.pdf']
---------------------split---------------------
['TPSP', 'D:\\Users\\88695\\Desktop\\經歷\\星展\\test.xlsx']
---------------------split---------------------
['SWOUT', 'C:\\Users\\USER\\Desktop\\星展實習程式\\']
---------------------split---------------------
['SPOUT', 'C:\\Users\\USER\\Desktop\\星展實習程式\\']
---------------------split---------------------
['MUREX', 'C:\\Users\\USER\\Desktop\\星展實習程式\\test.xlsx']
---------------------split---------------------
['DAYREPORT', 'C:\\Users\\USER\\Desktop\\星展實習程式\\']
---------------------split---------------------


<font size="6">SPOT交易單</font>

In [5]:
#----------------------broker:台北外匯 SPOT交易單(csv) 寫入excel--------------------
#讀台北外匯spot交易

buyerlist=[] 
sellerlist=[] 
amountlist=[] 
ratelist=[]
vdatelist=[] 
nolist=[]

TPFX=pd.read_excel(rf"{path['TPSP']}")
TPFX.fillna(value=0, inplace=True) 
for col in range (len(TPFX.columns)):
    for row in range (len(TPFX.iloc[:,col])): 
        if TPFX.iloc[row,col] =="NO":
            No_TP=list(TPFX.iloc[row:,col]) 
            #print (TPFX.iloc[:,col][row])
                       
        elif TPFX.iloc[row,col] =="RATE":
            Rate_T=list(TPFX.iloc[row:,col])
                       
        elif TPFX.iloc[row,col]=="SETTLE DATE :": 
            vdate=list(TPFX.iloc [row,col:])
            
        elif TPFX.iloc[row,col] =="BUYER":
            Buyer_TP=list(TPFX.iloc[row:,col])
                
        elif TPFX.iloc[row, col] =="SELLER":
            Seller_TP=list(TPFX.iloc[row:,col])
                
        elif "WE ARE PLEASED TO CONFIRM " in str(TPFX.iloc[row,col]):
            tradedate=TPFX.iloc[row,col][-10:] 
        
        elif TPFX.iloc[row,col] =="AMOUNT":
            Amount_TP=list(TPFX.iloc[row:,col]) 
                                          
        else:
            pass

In [8]:
# 轉換日期格式
tradedate=tradedate[-4:]+ "/" + tradedate
tradedate=tradedate.replace(f"{tradedate[-5:]}","")
                            
# 將各列資料加入list
for buyer in Buyer_TP:
    if isinstance(buyer, str):
        if buyer != "BUYER":
            buyerlist.append(buyer)

for seller in Seller_TP:
    if isinstance(seller, str):
        if seller != "SELLER":
            sellerlist.append(seller)

for amount in Amount_TP:
    if isinstance(amount, str):
        if amount != '0':
            amount = float(amount)
            amountlist.append(amount)

for rate in Rate_TP:
    if isinstance(rate, str):
        if rate != '0':
            ratelist.append(rate)

for n in No_TP:
    if not isinstance(n, str):
        if n != 0:
            nolist.append(str(n))

# 取出value日期
for vd in range(len(vdate)):
    if isinstance(vdate[vd], datetime.datetime):
        vdate = f"{vdate[vd]}"[0:10].replace("-", "/")

# print(Rate_TP)
# print(Buyer_TP)
# print(Seller_TP)
# print(vdate)
# print(Amount_TP)
# print(No_TP)
# print(buyerlist)
# print(sellerlist)
# print(amountlist)
# print(ratelist)
# print(vdate)
# print(nolist)
# print(tradedate)

In [9]:
# 利率取五位小數點轉換成字串
# 金額轉換字串
ratelist = ["{:.5f}".format(i) for i in ratelist]
amountlist = ["{:}".format(i) for i in amountlist]

# 根據上述資料整理成交易資訊以利對比
tradecode = []
for i in range(len(buyerlist)):
    code = ("TP" + buyerlist[i] + sellerlist[i] + "USD" + amountlist[i] + "TW1" + ratelist[i] + tradedate + vdate)
    tradecode.append(code)

# 寫入新的excel產生報表
wb = xlwt.Workbook()
taipeifx=wb.add_sheet('TPFX')
cosmos=wb.add_sheet('COSMOS')

taipeifx.write(0, 1, "NO")
taipeifx.write(0, 2, "BUY")
taipeifx.write(0, 3, "SELL")
taipeifx.write(0, 4, "CUR0")
taipeifx.write(0, 5, "AMOUNT")
taipeifx.write(0, 6, "CUR1")
taipeifx.write(0, 7, "RATE")
taipeifx.write(0, 8, "VALUEDATE")
taipeifx.write(0, 9, "TRA.DATE")
taipeifx.write(0, 10, "CODE")

row = 1
col = 1
step = 0
for i in range(len(buyerlist)):
    taipeifx.write(row + step, col, nolist[i])
    taipeifx.write(row + step, col + 1, buyerlist[i])
    taipeifx.write(row + step, col + 2, sellerlist[i])
    taipeifx.write(row + step, col + 3, "USD")
    taipeifx.write(row + step, col + 4, amountlist[i])
    taipeifx.write(row + step, col + 5, "TW1")
    taipeifx.write(row + step, col + 6, ratelist[i])
    taipeifx.write(row + step, col + 7, tradedate)
    taipeifx.write(row + step, col + 8, vdate)
    taipeifx.write(row + step, col + 9, tradecode[i])
    step += l
# print(tradecode)

In [10]:
#broker:cosmos SPOT 交易單(pdf)並寫入excel

name = [] #檔案名稱
pdfpath = [] #檔案路徑
costradecode = []

#找出檔案名稱以及路徑
for filename in glob.glob(rf"{path['COSP']}", recursive=True):
    y = os.path.split(filename)[1]
    name.append(y)
    pdfpath.append(filename)

# 讀取每一個pdf抓取所需資料    
for index in range(len(name)):
    temp = open(rf"{pdfpath[index]}", 'rb')
    reader = PdfFileReader(temp)
    text = ""
    reader.numPages
    text = reader.getPage(0).extractText()

    if text.count("\n") > 0:
        text = text.split("\n")
        row = l + index
        col = l
        step = 0
        
        for i in range(len(text)):
            if "OUR REF" in text[i]:
                code = text[i].split(':')[1]
                cosmos.write(row, col+step, code)
                step += 1
            elif "BUYER" in text[i]:
                buy = text[i].split(':')[1].strip()
                if "DBS BANK (TAIWAN), TAIPEI" == buy:
                    buy = 'DBSW'
                elif "DBS BANK LIMITED, TAIPEI" == buy:
                    buy = 'DBS'
                else:
                    pass                    
                cosmos.write(row, col+step, buy)
                step += 1
                
            elif "SELLER" in text[i]:
                sell = text[i].split(':')[1].strip()
                if "DBS BANK (TAIWAN), TAIPEI" == sell:
                    sell = 'DBSW'
                elif "DBS BANK LIMITED, TAIPEI" == sell:
                    sell = 'DBS'
                else:
                    pass
                    
                cosmos.write(row, col + step, sell)
                step += 1
                
            elif "AMOUNT" in text[i]:#amount  
                text[i].split(':')[1]
                amount = amount.replace(" ", "")
                amount = amount.replace("NTD", "TW1")
                cur0 = amount[:3]
                curl = amount[-3:]
                amount = amount[3:amount.index("A")]
                amount = str(float(amount.replace(",", "")) / 1000000)
                
                cosmos.write(row, col + step, cur0)
                step += 1
                cosmos.write(row, col + step, amount)
                step += 1
                cosmos.write(row, col + step, curl)
                step += 1
            
            elif "RATE" in text[i]:  # Rate
                text[i] = text[i].split(':')[1]
                rate=float(text[i])
                rate = "{:.5f}".format(rate)
                cosmos.write(row, col+step, rate)
                step += 1
                
            elif "VALUE DATE" in text[i]:  # Valuedate
                text[i] = text[i].split(':')[1]
                valuedate = cosmonth_change(text[i])
                cosmos.write(row, col+step, valuedate)
                step += 1
                
            elif "HERE COMES" in text[i]:
                tradedate = text[i][-11:]
                tradedate = cosmonth_change(tradedate)
            else:
                pass
        
        cosmos.write(row, col+step, tradedate)
        step += 1
                
        cosmos.write (row, col+step, "CO"+buy+sell+cur0+amount+cur1+rate+tradedate+valuedate)
        costradecode.append("CO"+buy+sell+cur0+amount+curl+rate+tradedate+valuedate)

    else:

        textindex=[]
        textindex.append(text.find("OUR REF"))
        textindex.append(text.find("BUYER"))
        textindex.append(text.find("SELLER"))
        textindex.append(text.find("AMOUNT"))
        textindex.append(text.find("RATE"))
        textindex.append(text.find("VALUE DATE"))
        textindex.append(text.find("USD PAYMENT"))
        text=list(text)
        for i in range(len(textindex)):
            text.insert(textindex[i]+i,"\n")
            text=''.join(text)
            text=text.split("\n")
            print(text)
            row=l+index
            col=l
            step=0
            
            for i in range(len(text)):
                #text[i] = text[i].split(':')[1]
                if "OUR REF" in text[i]:
                    code = text[i].split(':')[1]
                    cosmos.write(row, col+step, code)
                    step += 1

                elif "BUYER" in text[i]:
                    buy = text[i].split(':')[1].strip()
                    if "DBS BANK (TAIWAN), TAIPEI" == buy:
                        buy = 'DBSW'
                    elif "DBS BANK LIMITED, TAIPEI" == buy:
                        buy = 'DBS'
                    cosmos.write(row, col + step, buy)
                    step += 1

                elif "SELLER" in text[i]:
                    sell = text[i].split(':')[1].strip()
                    if "DBS BANK (TAIWAN), TAIPEI" == sell:
                        sell = 'DBSW'
                    elif "DBS BANK LIMITED, TAIPEI" == sell:
                        sell = 'DBS'
                    cosmos.write(row, col + step, sell)
                    step += 1

                elif "AMOUNT" in text[i]:  # amount
                    text[i] = text[i].split(':')[1]
                    amount = text[i].replace(" ", "")
                    amount = amount.replace("NTD", "TW1")
                    cur0 = amount[:3]
                    curl = amount[-3:]
                    amount = amount[3:amount.index("A")]
                    amount = str(float(amount.replace(",", "")) / 1000000)
                    print(amount)
                    print(cur0)
                    print(cur1)
                    cosmos.write(row, col + step, cur0)
                    step += 1
                    cosmos.write(row, col + step, amount)
                    step += 1
                    cosmos.write(row, col + step, curl)
                    step += 1

                elif "RATE" in text[i]:  # Rate
                    rate = float(text[i].split(':')[1])
                    rate = "{:.5f}".format(rate)
                    cosmos.write(row, col + step, rate)
                    step += 1

                elif "VALUE DATE" in text[i]:  # Valuedate
                    valuedate = text[i].split(':')[1].strip()
                    valuedate = cosmonth_change(valuedate)
                    cosmos.write(row, col + step, valuedate)
                    step += 1

                elif "HERE COMES" in text[i]:
                    tradedate = text[i][-11:]
                    tradedate = cosmonth_change(tradedate)
                    cosmos.write(row, col + step, tradedate)
                    step += 1
                else:
                    pass
            cosmos.write(row, col + step, tradedate)
            step += 1
        
        cosmos.write(row, col + step, "CO" + buy + sell + cur0 + amount + curl + rate + tradedate + valuedate)
        costradecode.append("CO" + buy + sell + cur0 + amount + curl + rate + tradedate + valuedate)

cosmos.write(0, 1, "NO")
cosmos.write(0, 2, "BUY")
cosmos.write(0, 3, "SELL")
cosmos.write(0, 4, "CUR0")
cosmos.write(0, 5, "AMOUNT")
cosmos.write(0, 6, "CUR1")
cosmos.write(0, 7, "RATE")
cosmos.write(0, 8, "VALUEDATE")
cosmos.write(0, 9, "TRA.DATE")
cosmos.write(0, 10, "CODE")

wb.save(rf'{path["SPOUT"]}{today}_SPOT.xls')
 


<font size="6">SWAP交易單</font>

In [11]:
#broker:台北外匯 SWAP 交易單(csv)並寫入excel

buyerlist=[]
sellerlist=[]
amountlist=[]
ratelist=[]
vdatelist=[]
nolist=[]
try:
    TPFXRB=pd.read_excel(rf"{path['TPSWRB']}")
    TPFXRB.filina(value=0,inplace=True)
    for col in range(len(TPFXRB.columns)):
        for row in range (len(TPFXRB.iloc[:,col])):   
            if TPFXRB.iloc[row,col] =="N0":
                No_TPRB =list(TPFXRB.iloc[row:,col])
               #print(TPFX.iloc[:,col][row])
            elif TPFXRB.iloc[row,col] =="SPOT RATE":
                Rate_TPRB=list(TPFXRB.iloc[row:,col])
            elif TPFXRB.iloc[row,col] =="SPOT VALUE":
                vdateRB=list(TPFXRB.iloc[row: ,col])
            elif TPFXRB.iloc[row,col] =="B/S":
                Buyer_TPRB=list(TPFXRB.iloc[row:,col])
            elif TPFXRB.iloc[row,col] =="S/B":
                Seller_TPRB=list(TPFXRB.iloc[row:,col])
            elif "WE ARE PLEASED TO CONFIRM " in str(TPFXRB.iloc[row,col]):
                tradedate=TPFXRB.iloc[row,col][-10:]
            elif TPFXRB.iloc[row,col] =="AMOUNT":
                Amount_TPRB=list(TPFXRB.iloc[row:, col])
            else:
                pass
            
    #轉換日期格式
    tradedate=tradedate[-4:]+"/"+tradedate
    tradedate=tradedate.replace(f"{tradedate[-5:]}","")

    for buyer in Buyer_TPRB:
        if type(buyer) == str:
            if buyer != "B/S":
                buyerlist.append(buyer)
        else:
            pass
        print(buyerlist)


    for seller in Seller_TPRB:
        if type(seller) == str:
            if seller != "S/B":
                sellerlist.append(seller)
        else:
            pass
            
    for amount in Amount_TPRB:
        if type(amount) != str:
            if amount != 0:
                amount=float(amount)
                amountlist.append(amount)
        else:
            pass



    for rate in Rate_TPRB:
        if type(rate) != str:
            if rate != 0:
                ratelist.append(rate)
        else:
            pass

    for n in No_TPRB:
        if type(n) != str:
            if n != 0:
                nolist.append(str(n))
        else:
            pass
        
    for vd in range(len(vdateRB)):
        if isinstance(vdateRB[vd],datetime,datetime) == True:
            vdateRB[vd]=f"{vdateRB[vd]}"[0:10].replace("-","/")
            vdatelist.append(vdateRB[vd])
        else:
            pass
except:
    pass


In [12]:
#------------------SUB交易單.................
try:
    TPFXSUB=pd.read_excel(rf"{path['TPSWSUB']}")
    TPFXSUB.fillna(value=0,inplace=True)
    for col in range(len(TPFXSUB.columns)):
        for row in range (len(TPFXSUB.iloc[:,col])):
            if TPFXSUB.iloc[row,col] =="N0":
                No_TPSUB =list(TPFXSUB.iloc[row:, col])
                #print(TPFX.iloc[:,col][row])
            elif TPFXSUB.iloc[row,col] =="SPOT RATE":
                Rate_TPSUB=list(TPFXSUB.iloc[row:,col])
            elif TPFXSUB.iloc[row,col] =="SPOT VALUE":
                vdateSUB=list(TPFXSUB.iloc[row:,col])
            elif TPFXSUB.iloc[row,col] =="B/S":
                Buyer_TPSUB=list(TPFXSUB.iloc[row:,col])
            elif TPFXSUB.iloc[row,col] =="S/B":
                Seller_TPSUB=list(TPFXSUB.iloc[row:,col])
            elif "WE ARE PLEASED TO CONFIRM " in str(TPFXSUB. iloc[row, col]):
                tradedate=TPFXSUB.iloc[row,col][-10:]
            elif TPFXSUB.iloc[row,col] =="AMOUNT":
                Amount_TPSUB=list(TPFXSUB.iloc[row:,col])
            else:
                pass

    #轉換日期格式        
    tradedate=tradedate[-4: ]+"/"+tradedate
    tradedate=tradedate.replace(f"{tradedate[-5: ]}","")
    
    for buyer in Buyer_TPSUB:
        if type(buyer) == str:
            if buyer != "B/S":
                buyerlist.append(buyer)
        else:
            pass

    for seller in Seller_TPSUB:
        if type(seller) == str:
            if seller != "S/B":
                sellerlist.append(seller)
        else:
            pass
        
    for amount in Amount_TPSUB:
        if type(amount) != str:
            if amount != 0:
                amount=float(amount)
                amountlist.append(amount)
        else:
            pass


    for rate in Rate_TPSUB:
        if type(rate) != str:
            if rate != 0:
                ratelist.append(rate)
        else:
            pass

    for n in No_TPSUB:
        if type(n) != str:
            if n != 0:
                nolist.append(str(n))
        else:
            pass

    for vd in range(len(vdateSUB)):
        if isinstance(vdateSUB[vd],datetime,datetime) == True:
            vdateSUB[vd]=f"{vdateSUB[vd]}"[0:10].replace("-","/") 
            vdatelist.append(vdateSUB[vd])
        else:
            pass
except:
       pass
    
ratelist=["{:.5f}".format(i) for i in ratelist]
amountlist=["{:}".format(i) for i in amountlist]
    

In [13]:
#寫入excel
wb=xlwt.Workbook()
taipeifx=wb.add_sheet('TPFX')
cosmos=wb.add_sheet('COSMOS')

taipeifx.write(0,1,"NO") 
taipeifx.write(0,2,"BUY")
taipeifx.write(0,3,"SELL")
taipeifx.write(0,4,"CUR0")
taipeifx.write(0,5,"AMOUNT")
taipeifx.write(0,6,"CUR1")
taipeifx.write(0,7,"RATE")
taipeifx.write(0,8,"VALUEDATE")
taipeifx.write(0,9,"TRA.DATE")
taipeifx.write(0,10,"CODE")

taipeifx.write_merge(0,0,12,15,"當日SWAP筆數",setStyle(colour=5,bold=True,horz=0x03))
taipeifx.write_merge(1,1,12,13,"SUB",setStyle(colour=5,bold=True,horz=0x03))
taipeifx.write_merge(2,2,12,13,xlwt.Formula('COUNTIF(C1:D65536,"DBSW")'),setStyle(colour=5,bold=True,horz=0x03))
taipeifx.write_merge(1,1,14,15,"RB",setStyle(colour=5,bold=True,horz=0x03))
taipeifx.write_merge(2,2,14,15,xlwt.Formula('COUNTIF(C1:D65536,"DBS")'),setStyle(colour=5,bold=True,horz=0x03))



swtradecode=[]
row=1
col-1
step=0
for i in range(len(buyerlist)):
    print(i)
    print(step)
    taipeifx.write(row+step,col,nolist[i])
    taipeifx.write(row+step,col+1,buyerlist[i])
    taipeifx.write(row+step,col+2,sellerlist[i])
    taipeifx.write(row+step,col+3,"USD")
    taipeifx.write(row+step,col+4,amountlist[i])
    taipeifx.write(row+step,col+5,"TW1")
    taipeifx.write(row+step,col+6,ratelist[step])
    taipeifx.write(row+step+1,col+6,ratelist[step+1])
    taipeifx.write(row+step,col+7,vdatelist[step])
    taipeifx.write(row+step+1,col+7,vdatelist[step+1])
    taipeifx.write(row+step,col+8,str(tradedate))

    code=("TP"+buyerlist[i]+sellerlit[i]+vdatelist[step]+ratelist[step]+"USD"+amountlist[i]+"TW1"+
          vdatel1st[step+1]+ratelist[step+1]+str(tradedate))
    swtradecode.append(code)
    taipeifx.write(row+step,col+9,code)
    step+=2


    #print(swtradecode)



In [14]:
#broker:cosmos SWAP 交易單(pdf)並寫入excel
name=[]
pdfpath=[]

for filename in glob.glob(rf"{path['COSW']}",recursive=True):
        y=os.path.split(filename)[1]
        name.append(y)
        pdfpath.append(filename)

rowstep=1
for index in range(len(name)):
    temp = open(rf"{pdfpath[index]}",'rb')
    reader=PdfFileReader(temp)
    reader.numPages
    text=reader.getPage(0).extractText()
    if text.count("\n")>0:
        #判斷單子類型
        if "CANCELLATION" in text:#取消
              continue
        elif "NEW TRANSACTION" in text:#新交易單
            #print(text)
            text0=text.split ('--------')[0] #近腳
            text0=text0.split('\n')#近腳
            textl=text.split ('--------')[1]#遠腳
            textl=textl.split ('\n' )#遠腳
              
            #近腳
            row=rowstep
            col=1
            step=0
            
            for i in range(len(text0)):
                if "OUR REF" in text0[i]:
                    #print(text0[i])
                    code= text0[i].split(':')[1]
                    cosmos.write(row,col+step,code)
                    step+=1
                    
                elif "BUYER" in text0[i]:
                    buy=text0[i].split(':' )[1]
                    buy=buy.strip()
                    if "DBS BANK (TAIWAN), TAIPEI" == buy:
                        buy='DBSW'
                    elif "DBS BANK LIMITED, TAIPEI" == buy:
                          buy ='DBS'
                    else:
                        pass
                    cosmos.write(row,col+step,buy)
                    #print(buy)
                    step+=1
                    
                elif "SELLER" in text0[i]:
                    #print(text0[i])
                    sell=text0[i].split(':')[1]
                    sell=sell.strip()
                    if "DBS BANK (TAIWAN), TAIPEI" == sell:
                        sell='DBSW'
                    elif "DBS BANK LIMITED, TAIPEI" == sell:
                        sell ='DBS'
                    else:
                        pass
                    cosmos.write(row,col+step,sell)
                    step+=1
                    
                elif "AMOUNT" in text0[i]:#amount
                    #print(text0[i])
                    amount=text0[i].split(':')[1]
                    amount=amount.replace(" ","" )
                    amount=amount.replace("NTD","TW1")
                    cur0=amount[:3]
                    cur1=amount[-3:]
                    amount=amount[3:amount.index("A")]
                    amount=str(float(amount.replace(",",""))/1000000)

                    cosmos.write(row,col+step,cur0)
                    #print(cur0)
                    step+=1
                    cosmos.write(row,col+step,amount)
                    #print(amount)
                    step+=1
                    cosmos.write(row,col+step,curl)
                    #print(curl)
                    step+=1


                elif "RATE" in text0[i]:#Rate
                    print(text0[i])
                    rate0 = text0[i].split(':')[1]
                    rate0=float(rate0)
                    rate0='{:.5f}'.format(rate0)
                    cosmos.write(row,col+step,rate0)
                    print(rate0)
                    step+=1
                    
                elif "1ST VAL DATE" in text0[i]:#valuedate
                    print(text0[i])
                    text0[i] = text0[i].split(':')[1]
                    vdate0=cosmonth_change(text0[i])
                    cosmos.write(row,col+step,vdate0)
                    print(vdate0)
                    step+=1
                    
                elif "HERE COMES" in text0[i]:
                    tradedate=text0[i][-11: ]
                    tradedate=cosmonth_change(tradedate)
                    
                else:
                    pass


            cosmos.write(row,col+step,tradedate)
            step+=1
            
            #遠腳
            for i in range(len(text1)):
                if "RATE" in text1[i]:#Rate
                    rate1 = text1[i].split(':')[1]
                    print(rate1)
                    rate1=float(rate1)
                    rate1='{:.5f}'.format(rate1)
                    cosmos.write(row+1,col+step-3,rate1)

                elif "2ND VAL DATE" in text1[i]:#valuedate
                    text1[i] = text1[i].split(':')[1]
                    vdate1=cosmonth_change(text1[i])
                    cosmos.write(row+1,col+step-2,vdate1)
                else:
                    pass

        elif "AMENDMENT" in text:
            text0=text.split("NTD PAYMENT",1)[0]
            text1=text.split("NTD PAYMENT",1)[1]
            text0=text0.split("\n")
            text1=textl.split("\n")
            
            #近腳
            row=rowstep
            col=1
            step=0
            for i in range(len(text0)):
                if "OUR REF" in text0[i]:
                    #print(text0[i])
                    code= text0[i].split(':')[1]
                    cosmos.write(row,col+step,code)
                    step+=1
                elif "BUYER" in text0[i]:
                    #print(text0[i])
                    buy=text0[i].split(':')[1]
                    buy=buy.strip()
                    if "DBS BANK (TAIWAN), TAIPEI" == buy:
                        buy='DBSW'
                    elif "DBS BANK LIMITED, TAIPEI" == buy:
                        buy ='DBS'
                    else:
                        pass
                    cosmos.write(row,col+step,buy)
                    #print(buy)
                    step+=l
                elif "SELLER" in text0[i]:
                    #print(text0[i])
                    sell=text0[i].split(':')[1]
                    sell=sell.strip()
                    if "DBS BANK (TAIWAN), TAIPEI" == sell:
                        sell='DBSW'
                    elif "DBS BANK LIMITED, TAIPEI" == sell:
                        sell ='DBS'  
                    else:
                        pass
                    cosmos.write(row,col+step,sell)
                    step+=1
                elif "AMOUNT" in text0[i]:#amount
                    #print(text0[i])
                    amount=text0[i].split(':')[1]
                    amount=amount.replace(" ","")
                    amount=amount.replace("NTD","TW1")
                    cur0=amount[:3]
                    cur1=amount[-3:]
                    amount=amount [3:amount.index("A")]
                    amount=str(float(amount.replace(",",""))/1000000)

                    cosmos.write(row,col+step,cur0)
                    #print(cur0)
                    step+=1
                    cosmos.write(row,col+step,amount)
                    #print(amount)
                    step+=1
                    cosmos.write(row,col+step,cur1)
                    #print(curl)
                    step+=1

                elif "HERE COMES" in text0[i]:
                    tradedate=text0[i][-11:]
                    tradedate=cosmonth_change(tradedate)

                elif "RATE" in text0[i]:#Rate
                    rate0 = text0[i].split(':')[1]
                    rate0=float(rate0)
                    rate0='{:.5f }'.format(rate0)
                    cosmos.write(row,col+step,rate0)
                    step+=1

                elif "1ST VAL DATE" in text0[i]:#valuedate
                    text0[i] = text0[i].split(':')[1]
                    vdate0=cosmonth_change(text0[i])
                    cosmos.write(row,col+step,vdate0)
                    step+=1
                else:
                    pass
            cosmos.write(row,col+step,tradedate)
            step+=1
            #遠腳    
            for i in range(len(text1)):
                if "RATE" in text1[i]:#Rate
                    rate1 = text1[i].split(':')[1]
                    rate1=float(rate1)
                    rate1='{:.5f}'.format(rate1)
                    cosmos.write(row+1,col+step-3,rate1)
                elif "2ND VAL DATE" in textl[i]:#valuedate
                    #print(textl[i])
                    text1[i] = text1[i].split(':')[1]
                    vdate1=cosmonth_change(text1[i])
                    cosmos.write(row+1,col+step-2,vdate1)
                else:
                    pass

                
        code='CO'+buy+sell+vdate0+rate0+'USD'+amount+'TWl'+vdate1+rate1+tradedate
        cosmos.wrlte(row,col+step,code)
        swtradecode.append(code)
        rowstep+=2
#如果pdf沒有分行       
    else:
        if "CANCELLATION" in text:
            continue
        elif "NEW TRANSACTION" in text:
            textindex=[]
            textindex=[]
            text=reader.getPage(0).extractText()
            text0=text.split("NTD PAYMENT",1)[0]
            text1=text.split("NTD PAYMENT",1)[1]


            textindex.append(text0.find("OUR REF"))
            textindex.append(text0.flnd("BUYER"))
            textindex.append(text0.find("SELLER"))
            textindex.append(text0.find("AMOUNT"))
            textindex.append(text0.flnd("RATE"))
            textindex.append(text0.flnd("1ST VAL DATE"))
            textindex.append(text0.flnd("USD PAYMENT"))
            text0=list(text0)
            for i in range(len(textindex)):
                text0.insert(textindex[i]+i,"\n")


            text0=''.join(text0)
            text0=text0.split("\n")


            textindex1.append(text1.find("RATE"))
            textindex1.append(text1.find("2ND VAL DATE"))
            textindex1.append(text1.find("USD PAYMENT"))


            text1=list(text1)
            for i in range(len(textindex1)):
                text1.insert(textindex1[i]+i,"\n")
            text1=''.join(text1)
            text1=text1.split("\n")
            #print(textl)
            row=rowstep
            col=1
            step=0
            #近腳
            for i in range(len(text0)):
                if "OUR REF" in text0[i]:
                    #print(text®[i])
                    code=text0[i].split(':')[1]
                    cosmos.write(row,col+step,code)
                    step+=1
                    
                elif "BUYER" in text0[i]:
                    #print(text0[i])
                    buy=text0[i].split(':')[1]
                    buy=buy.strip()
                    if "DBS BANK (TAIWAN), TAIPEI" == buy:
                        buy="DBSW"
                    elif "DBS BANK LIMITED, TAIPEI" == buy:
                        buy = "DBS"
                    else:
                        pass
                    cosmos.write(row,col+step,buy)
                    #print(buy)
                    step+=l
                    
                elif "SELLER" in text0[i]:
                    #print(text0[i])
                    sell=text0[i].split(':')[1]
                    sell=sell.strip()
                    if "DBS BANK (TAIWAN), TAIPEI" == sell:
                        sell="DBSW"
                    elif "DBS BANK LIMITED, TAIPEI" == sell:
                             sell ="DBS"
                    else:
                        pass
                    cosmos.write(row,col+step,sell)
                    step+=l
                    
                elif "AMOUNT" in text0[i]:#amount
                    #print(text0[i])
                    amount=text0[i].split(':')[1]
                    amount=amount.replace(" ","")
                    amount=amount.replace("NTD","TW1")
                    cur0=amount[:3]
                    cur1=amount[-3:]
                    amount=amount[3:amount.index("A")]
                    amount=str(float(amount.replace(","," "))/1000000)

                    cosmos.write(row,col+step,cur0)
                    #print(cur0)
                    step+=1
                    cosmos.write(row,col+step,amount)
                    #print(amount)
                    step+=1
                    cosmos.write(row,col+step,cur1)
                    #print(curl)
                    step+=1

                elif "RATE" in text0[i]:#Rate
                    #print(text0[i])
                    rate0 = text0[i].split(':')[1]
                    rate0=float(rate0)
                    rate0='{:.5f}'.format(rate0)
                    cosmos.write(row,col+step,rate0)
                    #print(rate0)
                    step+=1
                    
                elif "1ST VAL DATE" in text0[i]:#valuedate
                    # print(text0[i])
                    text0[i] = text0[i].split(':')[1]
                    vdate0=cosmonth_change(text0[i])
                    cosmos.write(row,col+step,vdate0)
                    #print(vdate0)
                    step+=1
                    
                elif "HERE COMES" in text0[i]:
                    tradedate=text0[i][-11:]
                    tradedate=cosmonth_change(tradedate)
                else:
                    pass

            cosmos.write(row,col+step,tradedate)
            step+=l

            #遠腳
            for i in range(len(text1)):
                if "RATE" in text1[i]:#Rate
                    rate1 = text1[i].split(":")[1]
                    #print(ratel)
                    rate1=float(rate1)
                    rate1='{:.5f}'.format(rate1)
                    cosmos.write(row+1,col+step-3,rate1)

                elif "2ND VAL DATE" in text1[i]:#valuedate
                    text1[i] = text1[i].split(':')[1]
                    vdate1=cosmonth_change(text1[i])
                    cosmos.write(row+1,col+step-2,vdate1)
                
                else:
                    pass

        elif "AMENDMENT" in text:
            textindex=[]
            textindex1=[]

            text0=text.split("NTD PAYMENT")[0]
            textindex.append(text0.find("OUR REF"))
            textindex.append(text0.find("BUYER"))
            textindex.append(text0.find("SELLER"))
            textindex.append(text0.find("AMOUNT"))
            textindex.append(text0.find("RATE"))
            textindex.append(text0.find("lST VAL DATE"))
            textindex.append(text0.find("USD PAYMENT"))
            text0=list(text0)
            for i in range(len(textindex)):
                text0.insert(textindex[i]+i,"\n")
            text0=''.join(text0)
            text0=text0.split("\n")
            
            text1=text.split("NTD PAYMENT")[1]
            textindex1.append(text1.find("RATE"))
            textindex1.append(text1.find("2ND VAL DATE"))
            textindex1.append(text1.find("USD PAYMENT"))
            textl=list(text1)
            for i in range(len(textindex1)):
                text1.insert(textindex1[i]+i,"\n")
                text1=''.join(text1)
                text1=text1.split("\n")



            #近腳
            row=rowstep
            col=1
            step=0
            for i in range(len(text0)):
                if "OUR REF" in text0[i]:
                    #print(text0[i])
                    code= text0[i].split(":")[1]
                    cosmos.write(row,col+step,code)
                    step+=1
                elif "BUYER" in text0[i]:
                    #print(text0[i])
                    buy=text0[i].split(":")[1]
                    buy=buy.strip()
                    if "DBS BANK (TAIWAN), TAIPEI" == buy:
                        buy="DBSW"
                    elif "DBS BANK LIMITED, TAIPEI" == buy:
                        buy ="DBS"
                    else:
                        pass
                    cosmos.write(row,col+step,buy)
                    #print(buy)
                    step+=1
                elif "SELLER" in text0[i]:
                    #print(text0[i])
                    sell=text0[i].split(':')[1]
                    sell=sell.strip()
                    if "DBS BANK (TAIWAN), TAIPEI" == sell:
                        sell="DBSW"
                    elif "DBS BANK LIMITED, TAIPEI" == sell:
                        sell="DBS"
                    else:
                        pass
                    cosmos.write(row,col+step,sell)
                    step+=1
                elif "AMOUNT" in text0[i]:#amount
                    #print(text0[i])
                    amount=text0[i].split(":")[1]
                    amount=amount.replace(" ","")
                    amount=amount.replace("NTD","TW1")
                    cur0=amount[:3]
                    cur1=amount[-3:]
                    amount=amount[3:amount.index("A")]
                    amount=str(float(amount.replace(",",""))/1000000)
                    cosmos.write(row,col+step+cur0)
                    #print(cur0)
                    step+=1
                    cosmos.write(row,col+step,amount)
                    #print(amount)
                    step+=1
                    cosmos.write(row,col+step,cur1)
                    #print(curl)
                    step+=1
                               
                elif "HERE COMES" in text0[i]:
                    tradedate=text0[i][-11:]
                    tradedate=cosmonth_change(tradedate)

                elif "RATE" in text0[i]:#Rate
                    rate0 = text0[i].split(':')[1]
                    rate0=float(rate0)
                    rate0='{:.5f}'.format(rate0)
                    cosmos.write(row,col+step,rate0)
                    step+=l
                
                elif "1ST VAL DATE" in text0[i] :#valuedate
                    #print(text0[i])
                    text0[i] = text0[i].split(":")[1]
                    #print(text0[i])
                    vdate0=cosmonth_change(text0[i])
                    cosmos.write(row,col+step,vdate0)
                    #print(vdate0)
                    step+=l
                else:
                    pass
                cosmos.write(row,col+step,tradedate)
                step+=1
                for i in range(len(text1)):
                    if "RATE" in text1[i]:#Rate
                        rate1= text1[i].split(":")[1]
                        rate1=float(rate1)
                        rate1='{:・5f}'.format(ratel)
                        cosmos.write(row+1,col+step-3,rate1)
                    elif "2ND VAL DATE" in text1[i]:#valuedate
                        #print(text1[i])
                        text1[i] = text1[i].split(":")[1]
                        vdate1=cosmonth_change(textl[i])
                        cosmos.write(row+1,col+step-2,vdate1)
                        #print(vdate1)
                    else:
                        pass

    code='CO'+buy+sell+vdate0+rate0+"USD"+amount+"TWl"+vdate1+rate1+tradedate
    cosmos.write(row,col+step,code)
    swrtradecode.append(code)
    rowstep+=2
        
print(swtradecode)
cosmos.write(0,1,"NO")
cosmos.write(0,2,"BUY")
cosmos.write(0,3,"SELL")
cosmos.write(0,4,"CUR0")
cosmos.write(0,5,"AMOUNT")
cosmos.write(0,6,"CUR1")
cosmos.write(0,7,"RATE")
cosmos.write(0,8,"VALUEDATE")
cosmos.write(0,9,"TRA.DATE")
cosmos.write(0,10,"CODE")

cosmos.write_merge(0,0,12,15,"當日 SWAP 筆數",setStyle(colour=5,bold=True,horz=0x03))
cosmos.write_merge(1,1,12,13,"SUB",setStyle(colour=5,bold=True,horz=0x03))
cosmos.write_merge(2,2,12,13,xlwt.Formula('COUNTIF(C1:D65536,"DBSW")'),setStyle(colour=5,bold=True,horz=0x03))
cosmos.write_merge(1,1,14,15,"RB",setStyle(colour=5,bold=True,horz=0x03))
cosmos.write_merge(2,2,14,15,xlwt.Formula('COUNTIF(C1:D65536,"DBS")'),setStyle(colour=5,bold=True,horz=0x03))

wb.save(rf'{path["SWOUT"]}{today}_SWAP.xls')





[]


<font size="6">murex系統交易單</font>

In [15]:
today=date.today()
today=today.strftime('%Y%m%d')

#murex系統SPOT交易單讀檔
sp=pd.read_excel(rf'{path["MUREX"]}',sheet_name="SPOT TODAY")
sp.fillna(value=0,inplace=True)

print(sp)
#將murex中銀行縮寫換成台北外匯的code字典建立
murextp=sp["Unnamed: 43"]
tpcode=sp["Unnamed: 44"]
dictp=dict(zip(murextp,tpcode))

#將murex中銀行縮寫換成cos的code字典建立
murexco=sp["Unnamed: 47"]
cocode=sp["Unnamed: 48"]
dicco=dict(zip(murexco,cocode))
print('.............here.................')

   Unnamed: 0  Unnamed: 1  Unnamed: 2  Unnamed: 3   NB  Counterparty  \
0         0.0         0.0         0.0         0.0  0.0           0.0   
1         0.0         0.0         0.0         0.0  0.0           0.0   

   Portfolio  TRN.DATE  B/S  CUR 0  ...  Unnamed: 39  Unnamed: 40  \
0        0.0       0.0  0.0    0.0  ...          0.0          0.0   
1        0.0       0.0  0.0    0.0  ...          0.0          0.0   

   Unnamed: 41  Unnamed: 42  Unnamed: 43  Unnamed: 44  Unnamed: 45  \
0          0.0          0.0            0            0          0.0   
1          0.0          0.0      MUREXTP       TPCODE          0.0   

   Unnamed: 46  Unnamed: 47  Unnamed: 48  
0          0.0            0            0  
1          0.0      MUREXCO       COCODE  

[2 rows x 49 columns]
.............here.................


In [16]:
#交易分類
counterparty=list(sp["Counterparty"])#對手
while 0 in counterparty:
    counterparty.remove(0)

Portfolio=list(sp["Portfolio"])
while 0 in Portfolio:
    Portfolio.remove(0)

tradedate=list(sp["TRN.DATE"])#交易日期
while 0 in tradedate:
    tradedate.remove(0)

bs=list(sp["B/S"])#買賣
while 0 in bs:
    bs.remove(0)

cur0=list(sp["CUR 0"])#幣別
while 0 in cur0:
    cur0.remove(0)

amount=list(sp["NOMINAL 0"])#金額
while 0 in amount:
    amount.remove(0)

cur1=list(sp["CUR 1"])#幣別
while 0 in cur1:
    cur1.remove(0)

rate=list(sp["RATE 0"])#利率
while 0 in rate:
    rate.remove(0)

#print(type(rate[2]))
valuedate=list(sp["Maturity"])#到期日
while 0 in valuedate:
    valuedate.remove(0)

broker=list(sp["broker destination 1st line"])#broker
while 0 in broker:
    broker.remove(0)

nb=list(sp["NB"])#編號
while 0 in nb:
    nb.remove(0)
    
#  print(nb)
#  print(len(counterparty))
#  print(len(tradedate))
#  print(len(bs))
#  print(len(cur0))
#  print(len(amount))
#  print(len(curl))
#  print(len(rate))


#tradedate日期轉換
for i in range(len(tradedate)):
    if isinstance(tradedate[i],datetime.datetime) == True:
        tradedate[i]=f"{tradedate[i]}"[0:10].repiace("-","/")
    else:
        pass

#valuedate日期轉換
for i in range(len(valuedate)):
    if isinstance(valuedate[i],datetime.datetime) == True:
        valuedate[i]=f"{valuedate[i]}"[0:10].replace("-","/")
    else:
        pass

#把利率變為小數點後4位
for i in range(len(rate)):
    rate[i]="{:.5f}".format(rate[i])


#金額轉換
for i in range(len(amount)):
    if amount[i] >0:
        amount[i]=f"{amount[i]/1000000}"
#print(amount)

#將對手名稱換成縮寫
for i in range (len(counterparty)):
    if broker[i] == "TAIPEI FX":
        if counterparty[i] in dictp:
            counterparty[i]=counterparty[i].replace(f"{counterparty[i]}",f"{dictp[counterparty[i]]}")
    elif broker[i] =='COSMOS FX':
        if counterparty[i] in dicco:
            counterparty[i]=counterparty[i].replace(f"{counterparty[i]}",f"{dicco[counterparty[i]]}")
#print(counterparty)
#將自己名稱換掉
for i in range(len(Portfolio)):
    if str(Portfolio[i])[0:2] == "DT":
         Portfolio[i] = "DBSW"
    elif str(Portfolio[i])[0:2] == "DB":
         Portfolio[i] = "DBS"
    else:
        print(f'{Portfolio[i]}_SPOT_not changed')

spmurexdeal=[]
for i in range(len(nb)):
    #台北外匯
    if broker[i] == "TAIPEI FX":
        if bs[i] == "Sell":#如果是賣方
            mucode=("TP"+counterparty[i]+Portfolio[i]+cur0[i]+amount[i]+cur1[i]+
                    str(rate[i])+str(tradedate[i])+str(valuedate[i]))
            spmurexdeal.append(mucode)
        elif bs[i] == "Buy":#如果是買方
            mucode=("TP"+Portfolio[i]+counterparty[i]+cur0[i]+amount[i]+cur1[i]+ 
                    str(rate[i])+str(tradedate[i])+str(valuedate[i]))
            spmurexdea1.append(mucode)
    #cosmos
    elif broker[i] =="COSMOS FX":
        if bs[i] == "Sell":#如果是賣方
            mucode=("CO"+counterparty[i]+Portfolio[i]+cur0[i]+amount[i]+cur1[i]+
                    str(rate[i])+str(tradedate[i])+str(valuedate[i]))
            spmurexdeal.append(mucode)
        elif bs[i] == "Buy":#如果是買方
            mucode=("CO"+Portfolio[i]+counterparty[i]+cur0[i]+amount[i]+cur1[i]+
                    str(rate[i])+str(tradedateiij)+str(valuedate[i]))
            spmurexdeal.append(mucode)
    #傳真交易
    elif broker[i] == "PH":
        spmurexdeal.append("PH")
    else:
        spmurexdeal.append("2")



#寫入excel
W=xlwt.Workbook()
SP=W.add_sheet('SPOT TODAY',cell_overwrite_ok=True)



SP.col(0).width=256*15
SP.col(3).width=256*15
SP.col(4).width=256*12
SP.col(5).width=256*12
SP.col(6).width=256*12
SP.col(7).width=256*15
SP.col(8).width=256*5
SP.col(9).width=256*5
SP.col(10).width=256*5
SP.col(11).width=256*5
SP.col(12).width=256*10
SP.col(13).width=256*15

style1=setStyle(horz=0X03,bold=True)
style2=setStyle(horz=0X03,colour=29,bold=True)

#表頭
SP.write(0,3, 'Status',style1)
SP.write(0,4,'NB',style1)
SP.write(0,5,'Broker',style1)
SP.write(0,6,'Counterparty',style1)
SP.write(0,7,'TRN.DATE',style1)
SP.write(0,8,'B/S',style1)
SP.write(0,9,'CUR 0',style1)
SP.write(0,10,'AMOUNT',style1)
SP.write(0,11,'CUR 1',style1)
SP.write(0,12,'RATE',style1)
SP.write(0,13,'VALUEDATE',style1)
SP.write(0,14,'CODE',style1)

SP.write_merge(0,0,0,1,'SPOT-MU筆數',setStyle(colour=26,bold=True))
SP.write(1,0,'COSMOS FX',setStyle(colour=26,bold=True))
SP.write(2,0,'TAIPEI FX',setStyle(colour=26,bold=True))
SP.write(3,0,'PH',setStyle(colour=26,bold=True))
SP.write(4,0,'TOTAL',setStyle(colour=26,bold=True))

SP.write(1,1,xlwt.Formula('COUNTIF(F1:F65536,$A$2)'),setStyle(colour=26,bold=True))
SP.write(2,1,xlwt.Formula('COUNTIF(F1:F65536,$A$3)'),setStyle(colour=26,bold=True))
SP.write(3,1,xlwt.Formula('COUNTIF(F1:F65536,$A$4)'),setStyle(colour=26,bold=True))
SP.write(4,1,xlwt.Formula('SUM(B2:B4)'),setStyle(colour=26,bold=True))

for row in range(len(nb)):
        col=4
        if row%2 == 0:
            SP.write(row+1,col,nb[row],style2)
            col+=1
            SP.write(row+1,col,broker[row],style2)
            col+=1
            SP.write(row+1,col,counterparty[row],style2)
            col+=1
            SP.write(row+1,col,tradedate[row],style2)
            col+=1
            SP.write(row+1,col,bs[row],style2)
            col+=1
            SP.write(row+1,col,cur0[row],style2)
            col+=1
            SP.write(row+1,col,amount[row],style2)
            col+=1
            SP.write(row+1,col,cur1[row],style2)
            col+=1
            SP.write(row+1,col,rate[row],style2)
            col+=1
            SP.write(row+1,col,valuedate[row],style2)
            col+=1
            SP.write(row+l,col,spmurexdeal[row])
        else :
            SP.write(row+1,col,nb[row],style1)
            col+=1
            SP.write(row+1,col,broker[row],style1)
            col+=1
            SP.write(row+1,col,counterparty[row],style1)
            col+=1
            SP.write(row+1,col,tradedate[row],style1)
            col+=1
            SP.write(row+1,col,bs[row],style1)
            col+=1
            SP.write(row+1,col,cur0[row],style1)
            col+=1
            SP.write(row+1,col,amount[row],style1)
            col+=1
            SP.write(row+1,col,cur1[row],style1)
            col+=1
            SP.write(row+l,col,rate[row],style1)
            col+=1
            SP.write(row+1,col,valuedate[row],style1)
            col+=1
            SP.write(row+1,col,spmurexdeal[row])

In [17]:
#murex系統SWAP交易單讀檔
sw=pd.read_excel(rf"{path['MUREX']}",sheet_name="SWAP TODAY")
sw.fillna(value=0,inplace=True)
brokerclass=sw.groupby("broker destination 1st line")
# print(type(brokerclass・size()))


#將murex中銀行縮寫換成台北外匯的code字典建立
murextp=sw["Unnamed: 43"]
tpcod=sw["Unnamed: 44"]
dictp=dict(zip(murextp,tpcod))

#將murex中銀行縮寫換成cos的code字典建立
murexco=sw["Unnamed: 47"]
cocode=sw["Unnamed: 48"]
dicco=dict(zip(murexco,cocode))


In [18]:
#交易分類
counterparty=list(sw["Counterparty"])#對手
Portfolio=list(sw["Portfolio"])
tradedate=list(sw["TRN.DATE"])#交易日期
bs=list(sw["B/S"])#買賣
cur0=list(sw["CUR 0"])#幣別
amount=list(sw["NOMINAL 0"])#金額
cur1=list(sw["CUR 1"])# 幣別
rate=list(sw["RATE 0"])#利率
valuedate=list(sw["Maturity"])#到期日
broker=list(sw["broker destination 1st line"])#broker
nb=list(sw["NB"])#編號

#將broker 換到近腳
for i in range(len(broker)-1):
    if broker[i+1] != float("nan"):
        r=broker[i+l]
        broker[i+1]=broker[i]
        broker[i]=r
    else:
        pass
    
#tradedate日期轉換
for i in range(len(tradedate)):
    if isinstance(tradedate[i],datetime.datetime)==True:
        tradedate[i]=f"{tradedate[i]}"[0:10].replace("-","/")
    else:
        pass

#tradedate日期轉換
for i in range(len(valuedate)):
    if isinstance(valuedate[i],datetime.datetime)==True:
        valuedate[i]=f"{valuedate[i]}"[0:10].replace("-","/")
    else:
        pass

#把利率變為小數點後4位
for i in range(len(rate)):
    rate[i]="{:.5f}".format(rate[i])

#金額轉换
for i in range(len(amount)):
    if amount[i]>0:
        amount[i]=f"{amount[i]/1000000}"

#將對手名稱换成缩寫
for i in range (len(counterparty)):
    #print(broker[i])
    if broker[i]=="TAIPEI FX":
        if counterparty[i]in dictp:
            counterparty[i]=counterparty[i].replace(f"{counterparty[i]}",f"{dictp[counterparty[i]]}")

    elif broker[i]=="COSMOS FX":
        if counterparty[i]in dicco:
            counterparty[i]=counterparty[i].replace(f"{counterparty[i]}",f"{dicco[counterparty[i]]}")

#將自己名稱换掉
for i in range(len(Portfolio)):
    if str(Portfolio[i])[0:2]=="DT":
        Portfolio[i] = "DBSW"
    elif str(Portfolio[i])[0:2]=="DB":
        Portfolio[i] = "DBS"
    else:
        print(f'{Portfolio[i]}_SWAP_not changed')


while 0 in nb:
    nb.remove(0)

swapmurexdeal=[]
step=0
for i in range(int((len(nb)/2))):
    if broker[step]=="TAIPEI FX":
        if bs[step]=="Sell":
            mucode=("TP"+counterparty[step]+Portfolio[step]+str(valuedate[step])+str(rate[step])+cur0[step]
                    +amount[step]+cur1[step]+str(valuedate[step+1])+str(rate[step+1])+str(tradedate[step]))
            swapmurexdeal.append(mucode)
        elif bs[step]=="Buy":
            mucode=("TP"+Portfolio[step]+counterparty[step]+str(valuedate[step])+str(rate[step])+cur0[step]
                    +amount[step]+cur1[step]+str(valuedate[step+1])+str(rate[step+1])+str(tradedate[step]))
            swapmurexdeal.append(mucode)
            
    elif broker[step]=='COSMOS FX':
        if bs[step]=="Sell":
            mucode=("Co"+counterparty[step]+Portfolio[step]+str(valuedate[step])+str(rate[step])+cur0[step]
                    +amount[step]+cur1[step]+str(valuedate[step+1])+str(rate[step+1])+str(tradedate[step]))
            swapmurexdeal.append(mucode)
        elif bs[step]=="Buy":
            mucode=("Co"+Portfolio[step]+counterparty[step]+str(valuedate[step])+str(rate[step])+cur0[step]
                    +amount[step]+cur1[step]+str(valuedate[step+1])+str(rate[step+1])+str(tradedate[step]))
            swapmurexdeal.append(mucode)
            
    elif broker[step]=='PH':
        swapmurexdeal.append("PH")

    else:
        swapmurexdeal.append("2")
    
    step+=2

#寫入excel
SW=W.add_sheet('SWAP TODAY',cell_overwrite_ok=True)
SW.col(0).width=256*15
SW.col(3).width=256*15
SW.col(4).width=256*12
SW.col(5).width=256*12
SW.col(6).width=256*12
SW.col(7).width=256*15
SW.col(8).width=256*5
SW.col(9).width=256*5
SW.col(10).width=256*5
SW.col(11).width=256*5
SW.col(12).width=256*10
SW.col(13).width=256*15

s=0
for row in range(len(nb)):
        co1=4
        if row%2 ==0:
                SW.write(row+1,col,nb[row],style2)
                co1+=1
                SW.write(row+1,col,broker[row],style2)
                col+=1
                SW.write(row+1,col,counterparty[row],style2)
                co1+=1
                SW.write(row+1,col,tradedate[row],style2)
                co1+=1
                SW.write(row+1,col,bs[row],style2)
                co1+=1
                SW.write(row+1,col,cur0[row],style2)
                co1+=1
                SW.write(row+1,col,amount[row],style2)
                co1+=1
                SW.write(row+1,col,cur1[row],style2)
                co1+=1
                SW.write(row+1,col,rate[row],style2)
                co1+=1
                SW.write(row+1,col,valuedate[row],style2)
                co1+=1
        else:#遠腳寫在第二行
                SW.write(row+1,col+8,rate[row],style1)
                co1+=1
                SW.write(row+1,col+8,valuedate[row],style1)
                co1+=1
step=0
row=1
for i in range(len(swapmurexdeal)):
    SW.write(row+step,14,swapmurexdeal[i])
    step+=2

#step=0
for i in range(len(nb)):
    if i%2 == 0:
        SW.write(i+1,15,Portfolio[i])
    #step+=2

first_col=SW.col(0)
first_col.width=256*15
SW.write(0,3,'Status',style1)
SW.write(0,4,'NB',style1)
SW.write(0,5,'Broker',style1)
SW.write(0,6,'Counterparty',style1)
SW.write(0,7,'TRN.DATE',style1)
SW.write(0,8,'B/S',style1)
SW.write(0,9,'CUR 0',style1)
SW.write(0,10,'AMOUNT',style1)
SW.write(0,11,'CUR 1',style1)
SW.write(0,12,'RATE',style1)
SW.write(0,13,'VALUEDATE',style1)
SW.write(0,14,'CODE')

SW.write_merge(0,0,0,1,'RB-SWAP-MU',setStyle(colour=26,bold=True))
SW.write(1,0,'COSMOS FX',setStyle(colour=26,bold=True))
SW.write(2,0,'TAIPEI FX',setStyle(colour=26,bold=True))
SW.write(3,0,'PH',setStyle(colour=26,bold=True))
SW.write(4,0,'REUTERS',setStyle(colour=26,bold=True))
SW.write(5,0,'TOTAL',setStyle(colour=26,bold=True))

SW.write(1,1,'=COUNTIFS(F1:F65536,$A$2,P1:P65536,"DBS"',setStyle(colour=26,bold=True,horz=0x03))
SW.write(2,1,'=COUNTIFS(F1:F65536,$A$3,P1:P65536,"DBS"',setStyle(colour=26,bold=True,horz=0x03))
SW.write(3,1,'=COUNTIFS(F1:F65536,$A$4,P1:P65536,"DBS"',setStyle(colour=26,bold=True,horz=0x03))
SW.write(4,1,'=COUNTIFS(F1:F65536,$A$5,P1:P65536,"DBS"',setStyle(colour=26,bold=True,horz=0x03))

SW.write(5,1,xlwt.Formula('SUM(B2:B5)'),setStyle(colour=26,bold=True,horz=0x03))

SW.write_merge(7,7,0,1,'SUB-SWAP-MU',setStyle(colour=26,bold=True))
SW.write(8,0,'COSMOS FX',setStyle(colour=26,bold=True))
SW.write(9,0,'TAIPEI FX',setStyle(colour=26,bold=True))
SW.write(10,0,'PH',setStyle(colour=26,bold=True))
SW.write(11,0,'TOTAL',setStyle(colour=26,bold=True))

SW.write(8,1,'=COUNTIFS(F1:F65536,$A$9,P1:P65536,"DBSW"',setStyle(colour=26,bold=True,horz=0x03))
SW.write(9,1,'=COUNTIFS(F1:F65536,$A$10,P1:P65536,"DBSW"',setStyle(colour=26,bold=True,horz=0x03))
SW.write(10,1,'=COUNTIFS(F1:F65536,$A$11,P1:P65536,"DBSW"',setStyle(colour=26,bold=True,horz=0x03))
SW.write(11,1,xlwt.Formula('SUM(B9:B11)'),setStyle(colour=26,bold=True,horz=0x03))



0.0_SWAP_not changed


In [19]:
#match交易單swap
swtp=pd.read_excel(rf"{path['SWOUT']}{today}_SWAP.xls",sheet_name="TPFX")
swco=pd.read_excel(rf"{path['SWOUT']}{today}_SWAP.xls",sheet_name="COSMOS")

swtpcode=list(swtp['CODE'])
swtpnb=list(swtp['NO'])
swcocode=list(swco['CODE'])
swconb=list(swco['NO'])

SW.write(13,0,'COSMOS-SWAP',setStyle(colour=26,bold=True))
SW.write(13,1,len(swconb)/2,setStyle(colour=26,bold=True))
SW.write(14,0,'COSMOS',setStyle(colour=26,bold=True))
SW.write(14,1,xlwt.Formula('IF(B2+B9=B14,"MATCH",IF(B14>B2+B9,"多"&B14-B2-B9&"筆","少"&B2+B9-B14&"筆"))'),setStyle(colour=26,bold=True))
SW.write(16,0,'TPFX-SWAP',setStyle(colour=26,bold=True))
SW.write(16,1,len(swtpnb)/2,setStyle(colour=26,bold=True))
SW.write(17,0,'TPFX',setStyle(colour=26,bold=True))
SW.write(17,1,xlwt.Formula('IF(B3+B10=B17,"MATCH",IF(B17>B3+B10,"多"&B17-B3-B18&"筆","少"&B3+B10-B17&"筆"))'),setStyle(colour=26,bold=True))

swnm=[]
for index in range(len(swapmurexdeal)):
    if swapmurexdeal[index]in swtpcode:
        tp=swtpcode.index(swapmurexdeal[index])
        print(f'{swtpcode.index(swapmurexdeal[index])}_0')
        swtpcode[tp]="checked"
        SW.write(index*2+1,3,swtpnb[tp],setStyle(horz=0X03,bold=True))
        swtpnb[tp]="checked"

    elif swapmurexdeal[index]in swcocode:
        co=swcocode.index(swapmurexdeal[index])
        swcocode[co]="checked"
        SW.write(index*2+1,3,swconb[co],setstyle(horz=0X03,bold=True))
        swconb[co]="checked"
    elif swapmurexdeal[index] == "PH":
        print("PH")
        SW.write(index*2+1,3,'PH',setStyle(horz=0X03,colour=44,bold=True))
    else:
        print(index)
        print(f'{swapmurexdeal[index]}_sw_not found')
        SW.write(index*2+1,3,'Pls Check',setStyle(horz=0X03,colour=43,fcolour=2,bold=True))
         
print(swconb[1])
print(type(swconb[1]))
print(swtpnb)
print(counterparty)
for nb in swconb:
    if type(nb) == str:
        if nb != 'checked':
            swnm.append(nb)

for nb in swtpnb:
    if type(nb)==float:
        if nb >0:
            swnm.append(int(nb))

SW.write(19,0,'PLS_CHECK',setStyle(colour=26,bold=True))
SW.write(20,0,str(swnm),setStyle(colour=26,bold=True))


nan
<class 'float'>
[nan, nan]
[0.0]


In [20]:
#match交易單spot
sptp=pd.read_excel(rf"{path['SPOUT']}{today}_SPOT.xls",sheet_name="TPFX")
spco=pd.read_excel(rf"{path['SPOUT']}{today}_SPOT.xls",sheet_name="COSMOS")

sptpcode=list(sptp['CODE'])
sptpnb=list(sptp['NO'])
spcocode=list(spco['CODE'])
spconb=list(spco['NO'])

SP.write(6,0,'COSMOS-SPOT',setStyle(colour=26,bold=True))
SP.write(6,1,len(spconb),setStyle(colour=26,bold=True))
SP.write(7,0,'COSMOS',setStyle(colour=26,bold=True))
SP.write(7,1,xlwt.Formula('IF(B2=B7,"MATCH",IF(B7>B2,"多"&B7-B2&"筆","少"&B2-B7&"筆"))'),setStyle(colour=26,bold=True))

SP.write(9,0,'TPFX-SPOT',setStyle(colour=26,bold=True))
SP.write(9,1,len(sptpnb),setStyle(colour=26,bold=True))
SP.write(10,0,'TPFX',setStyle(colour=26,bold=True))
SP.write(10,1,xlwt.Formula('IF(B3=B16,"MATCH",IF(B10>B3,"多"&B10-B3&"筆","少"&B3-B10&"筆"))'),setStyle(colour=26,bold=True))

                           
spnm=[]
for index in range(len(spmurexdeal)):
    if spmurexdeal[index]in sptpcode:
        tp=sptpcode.index(spmurexdeal[index])
        print(f'{sptpcode.index(spmurexdeal[index])}_sptpcode_index')
        sptpcode[tp]="checked"
        SP.write(index+1,3,sptpnb[tp],setStyle(horz=0X03,bold=True))
        sptpnb[tp]="checked"

    elif spmurexdeal[index]in spcocode:
        co=spcocode.index(spmurexdeal[index])
        spcocode[co]="checked"
        SP.write(index+1,3,spconb[co],setStyle(horz=0X03,bold=True))
        spconb[co]="checked"
        
    elif spmurexdeal[index] == "PH":
        print("PH")
        SP.write(index+1,3,'PH',setStyle(horz=0X03,colour=44,bold=True))
        
    else:
        print(f'{spmurexdeal[index]}_sp_not found')
        SP.write(index+1,3,'Pls Check',setstyle(horz=0X03,colour=43,fcolour=2,bold=True))

        
for nb in sptpnb:
    if nb != "checked":
        spnm.append(nb)

for nb in spconb:
    if nb != "checked":
        spnm.append(nb)

SP.write(12,0,'PLS_CHECK',setStyle(colour=26,bold=True))
SP.write(13,0,str(spnm),setStyle(colour=26,bold=True))


In [21]:
#--SWAP 交易單產生報表-- 包含checker需要資訊
chsw=pd.read_excel(rf"{path['MUREX']}",sheet_name="SWAP DAY END CHECK")
chsw.fillna(value="",inplace=True)
mp=pd.read_excel(rf"{path['MUREX']}",sheet_name="mapping")
print(chsw)
print(mp['Murex Label'])
mplist=list(mp['Murex Label'])

chspout=W.add_sheet('SPOT DAY END CHECK',cell_overwrite_ok=True)
chswout=W.add_sheet('SWAP DAY END CHECK',cell_overwrite_ok=True)
mpout=W.add_sheet('mapping',cell_overwrite_ok=True)
for col in range(len(chsw.columns)):
    for row in range (len(chsw.iloc[:,col])):
        chswout.write(row,col,chsw.iloc[row,col])

chswout.write(0,0,xlwt.Formula("TODAY()"))
chswout.write(1,2,xlwt.Formula("COUNTA(C5:C65536)/2"))
chswout.write(1,3,xlwt.Formula('COUNTIF(F5:F65536,"DEAD")/2'))
chswout.write(1,4,'=COUNTIFS(AI5:AI65536,"Others",AJ5:AJ65536,"1")/2')
chswout.write(1,5,xlwt.Formula('COUNTIF(G5:G65536,"<"&A1)/2'))
chswout.write(1,6,xlwt.Formula("C2-D2-E2-F2"))
chswout.write(1,7,xlwt.Formula('''IF('SWAP TODAY'!B6='SWAP DAY END CHECK'!G2,"MATCH","CHECK")'''),setStyle(horz=0X03,fcolour=2,bold=True))

mpout.write(0,3,'Murex Label')
for row in range(len(mplist)):
    mpout.write(row+1,3,mplist[row])

#最多可以寫200筆交易
for row in range(200):
    f=f'IF(COUNTIF(mapping!D1:D65536,E{row+5})>0,"Interbank","Others")'
    e=f'IF(F{row+5}="LIVE",1,0)'
    chswout.write(row+4,34,xlwt.Formula(f))
    chswout.write(row+4,35,xlwt.Formula(e))

#--SPOT 交易單產生報表-- 包含checker需要資訊   
chsp=pd.read_excel(rf"{path['MUREX']}",sheet_name="SPOT DAY END CHECK")
chsp.fillna(value="",inplace=True)

for col in range(len(chsp.columns)):
    for row in range (len(chsp.iloc[:,col])):
        chspout.write(row,col,chsp.iloc[row,col])

chspout.write(1,2,xlwt.Formula("COUNTA(C5:C65536)"))
chspout.write(1,3,xlwt.Formula('COUNTIF(F5:F65536,"DEAD")'))
chspout.write(1,4,'=COUNTIFS(AI5:AI65536,"Others",AJ5:AJ65536,"1")')
chspout.write(1,5,xlwt.Formula("C2-D2-E2"))
chspout.write(1,6,xlwt.Formula('''IF('SPOT TODAY'!B5='SPOT DAY END CHECK'!F2,"MATCH","CHECK")'''),setStyle(horz=0X03,fcolour=2,bold=True))

for row in range(200):
    f=f'IF(COUNTIF(mapping!D1:D65536,E{row+5})>0,"Interbank","Others")'
    e=f'IF(F{row+5}="LIVE",1,0)'
    chspout.write(row+4,34,xlwt.Formula(f))
    chspout.write(row+4,35,xlwt.Formula(e))

W.save(rf'{path["DAYREPORT"]}{today}.xls')

Empty DataFrame
Columns: []
Index: []
Series([], Name: Murex Label, dtype: object)
